# Comparativa de métodos

Cálculo de métricas y evolución del capital para distintas estrategias.


In [ ]:
import pandas as pd
import numpy as np
import joblib
import yfinance as yf
import matplotlib.pyplot as plt
from src import config as cfg


In [ ]:
# ─── Cargar series de retornos ───────────────────────────────────────
paths = {
    'Markowitz': cfg.RESULT / 'backtest_markowitz.pkl',
    'Ridge': cfg.RESULT / 'backtest_ridge.pkl',
    'LSTM-5d+VIX': cfg.RESULT / 'backtest_lstm5d.pkl',
}

def load_returns(path):
    res = joblib.load(path)
    if isinstance(res, pd.DataFrame) and 'ret_neto' in res.columns:
        serie = res.set_index('fecha')['ret_neto']
    elif isinstance(res, dict) and 'retorno' in res:
        serie = res['retorno']
    else:
        raise ValueError('Formato desconocido')
    return serie

series = {name: load_returns(fp) for name, fp in paths.items()}

# ─── SPY como benchmark ──────────────────────────────────────────────
start = min(s.index.min() for s in series.values())
spy = yf.download('SPY', start=start, progress=False)['Adj Close']
ret_spy = np.log(spy / spy.shift(1)).dropna()

calendar = ret_spy.index
for k in series:
    series[k] = series[k].reindex(calendar).fillna(0)
series['SPY'] = ret_spy
df_all = pd.DataFrame(series)


In [ ]:
# ─── Gráfico de retornos acumulados ──────────────────────────────────
wealth = (1 + df_all).cumprod()
ax = wealth.plot(figsize=(10,4))
ax.set_title('Evolución del capital')
ax.set_ylabel('Multiplicador')
ax.grid(True)
plt.show()


In [ ]:
# ─── Métricas de rentabilidad ───────────────────────────────────────
def stats(r):
    ann = np.sqrt(252)
    wealth = (1 + r).cumprod()
    total = wealth.iloc[-1] - 1
    vol   = r.std(ddof=1) * ann
    sharpe= r.mean() / r.std(ddof=1) * ann
    maxdd = (wealth.cummax() - wealth).max()
    return pd.Series([total, vol, sharpe, maxdd],
                     index=['Total return','Volatility','Sharpe','MaxDD'])

metrics = df_all.apply(stats).T.round(3)
display(metrics)


In [ ]:
# ─── Matriz de correlaciones ─────────────────────────────────────────
corr = df_all.drop(columns='SPY').corr().round(2)
display(corr)


In [ ]:
# ─── Diagrama de flujo ───────────────────────────────────────────────
from graphviz import Digraph
g = Digraph()
g.edge('Datos', 'Modelo')
g.edge('Modelo', 'Optimizador')
display(g)
